In [1]:
# !pip install cfgrib s3fs h5netcdf h5pyd

In [2]:
import io
from datetime import date, timedelta, datetime

In [3]:
# Constants for creating the full URL
source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com"
sector = "conus"
start_time = "2022-06-01 00:00"
fmt = "%Y-%m-%d %H:%M"
time1 = datetime.strptime(start_time, fmt)
day_date=time1.strftime("%Y%m%d")
cycle = 12          # hour
forecast_hour = 0   # offset from cycle time
product = "wrfsfcf" # 2D surface levels

# Put it all together
file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
grib_url=source_url+"/hrrr."+day_date+"/conus/"+file_path

print(grib_url)

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20220601/conus/hrrr.t12z.wrfsfcf00.grib2


In [6]:
# Setup
import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import io
import requests
import s3fs
import subprocess
from datetime import datetime

start_time = "2022-06-01 00:00"
end_time = "2022-06-30 00:00"

* https://registry.opendata.aws/noaa-hrrr-pds/

* https://spire.com/tutorial/spire-weather-tutorial-intro-to-processing-grib2-data-with-python/

* https://github.com/microsoft/AIforEarthDataSets/blob/main/data/noaa-hrrr.md

* https://nbviewer.org/github/microsoft/AIforEarthDataSets/blob/main/data/noaa-hrrr.ipynb

* https://github.com/ecmwf/cfgrib/issues/63

In [7]:
# Read in list of RAWS Stations
df = pd.read_csv("raws_stations_WA.csv")

# Filter to those with complete fmda data
df = df[(df[['air_temp', 'relative_humidity', 'precip_accum',
       'fuel_moisture', 'wind_speed', 'solar_radiation']]==1).sum(axis=1)==6]

# Get list of coords
points = list(df[["lon","lat"]].itertuples(index=False,name=None))
names = np.unique(df['STID'])

print(df.shape)

(40, 9)


In [8]:
from data_funcs import download_grib

Test one download

In [9]:
tempfile, url = download_grib(
    source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com",
    time = start_time,
    model = "wrfsfcf",
    dest_dir =  "data" # destination subdirectory for url content
)

wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20220601/conus/hrrr.t00z.wrfsfcf00.grib2 -O data/hrrr_202206010.grib2
The file 'data/hrrr_202206010.grib2' was successfully downloaded.


In [10]:
tempfile, url = download_grib(
    source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com",
    time = end_time,
    model = "wrfsfcf",
    dest_dir =  "data" # destination subdirectory for url content
)

wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20220630/conus/hrrr.t00z.wrfsfcf00.grib2 -O data/hrrr_202206300.grib2
The file 'data/hrrr_202206300.grib2' was successfully downloaded.


In [15]:
ds=xr.open_dataset(
    tempfile,
    filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2}
)

In [ ]:
os.remove(tempfile)

---
---

Get time range by looping through dates and: 

* temporarily downloading grib file at that date
* extract coords from temp file to build time series
* delete tempfile before iterating

In [ ]:
end_time = "2022-06-01 05:00"
fmt = "%Y-%m-%d %H:%M"
time1 = datetime.strptime(start_time, fmt)
time2 = datetime.strptime(end_time, fmt)
dates = pd.date_range(start=time1,end=time2, freq="1H") # Series of dates

In [ ]:
# Initialize matrix of time series obs
# (x, y, z) dims == (ntime, ncoords, nvars)
vars=["rh", "temp", "solar1", "solar2"]
nvars=len(vars)
obs_mat=np.zeros((dates.shape[0],len(points), nvars), np.float64)

In [ ]:
for d in dates:
    time=d.strftime("%Y-%m-%d %H:%M")
    # print(time)
    
    tempfile = download_grib(
        source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.",
        time = time,
        model = "t18z.wrfsubhf",
        dest_dir =  "data" # destination subdirectory for url content
    )
    
    

    ### manipulate tempfile

    os.remove(tempfile)

In [ ]:

time = dates[1].strftime("%Y-%m-%d %H:%M")
    
tempfile = download_grib(
    source_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.",
    time = time,
    model = "t18z.wrfsubhf",
    dest_dir =  "data" # destination subdirectory for url content
)

# manipulate tempfile

os.remove(tempfile)

In [ ]:
# gather_hrrr_time_range(
#     start=start_time,
#     end=end_time
# )